Grupo conformado por:
Edgar Andrés García Hernández - 200512532
John Pablo Calvo López - 201726690

# Exercise 04

# Fraud Detection

## Introduction

- Fraud Detection Dataset from Microsoft Azure: [data](http://gallery.cortanaintelligence.com/Experiment/8e9fe4e03b8b4c65b9ca947c72b8e463)

Fraud detection is one of the earliest industrial applications of data mining and machine learning. Fraud detection is typically handled as a binary classification problem, but the class population is unbalanced because instances of fraud are usually very rare compared to the overall volume of transactions. Moreover, when fraudulent transactions are discovered, the business typically takes measures to block the accounts from transacting to prevent further losses. 

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [2]:
import zipfile
with zipfile.ZipFile('../datasets/fraud_detection.csv.zip', 'r') as z:
    f = z.open('15_fraud_detection.csv')
    data = pd.io.parsers.read_table(f, index_col=0, sep=',')

In [3]:
data.head()

,accountAge,digitalItemCount,sumPurchaseCount1Day,sumPurchaseAmount1Day,sumPurchaseAmount30Day,paymentBillingPostalCode - LogOddsForClass_0,accountPostalCode - LogOddsForClass_0,paymentBillingState - LogOddsForClass_0,accountState - LogOddsForClass_0,paymentInstrumentAgeInAccount,ipState - LogOddsForClass_0,transactionAmount,transactionAmountUSD,ipPostalCode - LogOddsForClass_0,localHour - LogOddsForClass_0,Label
0,2000,0,0,0.00,720.25,5.064533,0.421214,1.312186,0.566395,3279.574306,1.218157,599.00,626.164650,1.259543,4.745402,0
1,62,1,1,1185.44,2530.37,0.538996,0.481838,4.401370,4.500157,61.970139,4.035601,1185.44,1185.440000,3.981118,4.921349,0
2,2000,0,0,0.00,0.00,5.064533,5.096396,3.056357,3.155226,0.000000,3.314186,32.09,32.090000,5.008490,4.742303,0
3,1,1,0,0.00,0.00,5.064533,5.096396,3.331154,3.331239,0.000000,3.529398,133.28,132.729554,1.324925,4.745402,0
4,1,1,0,0.00,132.73,5.412885,0.342945,5.563677,4.086965,0.001389,3.529398,543.66,543.660000,2.693451,4.876771,0


In [4]:
data.shape, data.Label.sum(), data.Label.mean()

((138721, 16), 797, 0.0057453449730033666)

In [5]:
X = data.drop(['Label'], axis=1)
y = data['Label']

# Exercise 04.1

Estimate a Logistic Regression

Evaluate using the following metrics:
* Accuracy
* F1-Score
* F_Beta-Score (Beta=10)

Comment about the results

In [6]:
np.random.seed(1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [7]:
logreg=LogisticRegression()
logreg.fit(X_train,y_train)
y_pred_class=logreg.predict(X_test)

from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))
print(metrics.f1_score(y_test,y_pred_class))
print(metrics.fbeta_score(y_test,y_pred_class,beta=10))

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_class)

0.9939736455119518
0.0
0.0


C:\Users\AndresGarcia\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


array([[34472,     0],
       [  209,     0]], dtype=int64)

In [11]:
# Se observó que el modelo tiene un serio problema con la sensibilidad. Lo anterior se debe en parte al desbalance en la muestra

# Exercice 04.2

Under-sample the negative class using random-under-sampling

Which is parameter for target_percentage did you choose?
How the results change?

**Only apply under-sampling to the training set, evaluate using the whole test set**

In [105]:
def UnderSampling(X, y, target_percentage=0.5, seed=None):
    # Assuming minority class is the positive
    n_samples = y.shape[0]
    n_samples_0 = (y == 0).sum()
    n_samples_1 = (y == 1).sum()

    n_samples_0_new =  n_samples_1 / target_percentage - n_samples_1
    n_samples_0_new_per = n_samples_0_new / n_samples_0

    filter_ = y == 0

    np.random.seed(seed)
    rand_1 = np.random.binomial(n=1, p=n_samples_0_new_per, size=n_samples)
    
    filter_ = filter_ & rand_1
    filter_ = filter_ | (y == 1)
    filter_ = filter_.astype(bool)
    
    return X[filter_], y[filter_]

X_u,y_u=UnderSampling(X_train,y_train,0.433,1)

logreg=LogisticRegression()
logreg.fit(X_u,y_u)
y_pred_class2=logreg.predict(X_test)

print(metrics.accuracy_score(y_test, y_pred_class2))
print(metrics.f1_score(y_test,y_pred_class2))
print(metrics.fbeta_score(y_test,y_pred_class2,beta=10))

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_class2)

0.7244600790057957
0.02150317427810772
0.34819581705355085


array([[25020,  9452],
       [  104,   105]], dtype=int64)

Se eligió un porcentaje de 43.3% para el porcentaje objetivo. Lo anterior se debió a una meta particular.
Específicamente, se buscó identificar al menos el 50% de los fraudes minimizando el número de falsos positivos.
Este número de fraudes a identificar con el modelo se eligió como política externa, buscando garantizar un mínimo de sensibilidad.
Valores más altos para la división de la muestra habrían aumentado el número de falsos positivos de forma desproporcionada.
Cabe mencionar, por otro lado, que una división de la muestra al 30% arrojaba el valor máximo para el puntaje F1 y alta precisión.
Sin embargo, detección de fraudes con esa división era muy baja (apenas 31 de los 209 en la muestra de prueba).

# Exercice 04.3

Now using random-over-sampling

Debido a problemas con el código mostrado en clase, el cual estaba generando NA para la variable dependiente al hacer el oversampling, dado un resultado np.isnan(y_o).any()=True, se utilizó el paquete imblearn para rebalancear la muestra.
http://contrib.scikit-learn.org/imbalanced-learn/stable/over_sampling.html

In [181]:
from imblearn.over_sampling import RandomOverSampler
ros1=RandomOverSampler(ratio=.79, random_state=1)
# En esta función, "ratio" no opera en forma porcentual sino como una tasa
# Es decir, un ratio de 1 hará que la muestra con oversampling tenga participaciones iguales para ambas categorías
# Un ratio 1 es equivalente a poner 0.5 en target_percentage para la función anterior
X_o,y_o=ros1.fit_sample(X_train,y_train)

logreg.fit(X_o,y_o)
y_pred_class3=logreg.predict(X_test)

print(metrics.accuracy_score(y_test, y_pred_class3))
print(metrics.f1_score(y_test,y_pred_class3))
print(metrics.fbeta_score(y_test,y_pred_class3,beta=10))

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_class3)

C:\Users\AndresGarcia\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


0.7844641157982757
0.02808477441164998
0.38432809527165107


array([[27098,  7374],
       [  101,   108]], dtype=int64)

Se eligió un ratio de 0.79, equivalente a un porcentaje de 39.5% para el objetivo. Nuevamente, se buscó identificar al menos el 50% de los fraudes minimizando el número de falsos positivos. Y este fue el ratio más bajo con el cual se cumplió esta meta. Nuevamente, valores más altos para el ratio disminuían los indicadores del modelo, al tiempo que ratios más bajos aumentaban la precisión y las F, a costa de una menor sensibilidad a fraudes.

# Exercice 04.4*
Evaluate the results using SMOTE

Which parameters did you choose?

Nuevamente, hubo problemas con el código mostrado en clase. Lo anterior se debió a problemas con las variables dentro de la función. Por ello, nuevamente se utilizó imblearn.

In [201]:
from imblearn.over_sampling import SMOTE
smote1=SMOTE(ratio=0.757, random_state=1)
# En esta función, "ratio" no opera en forma porcentual sino como una tasa
# Es decir, un ratio de 1 hará que la muestra con oversampling tenga participaciones iguales para ambas categorías
# Un ratio 1 es equivalente a poner 0.5 en target_percentage para la función anterior

X_s,y_s=smote1.fit_sample(X_train,y_train)

logreg.fit(X_s,y_s)
y_pred_class4=logreg.predict(X_test)

print(metrics.accuracy_score(y_test, y_pred_class4))
print(metrics.f1_score(y_test,y_pred_class4))
print(metrics.fbeta_score(y_test,y_pred_class4,beta=10))

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_class4)

C:\Users\AndresGarcia\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


0.7809463394942475
0.0281437891774338
0.3897151676722323


array([[26974,  7498],
       [   99,   110]], dtype=int64)

Se eligió un ratio de 0.757, equivalente a un porcentaje de 37.85% para el objetivo. Nuevamente, se buscó identificar al menos el 50% de los fraudes minimizando el número de falsos positivos. Y este fue el ratio más bajo con el cual se cumplió esta meta. Nuevamente, valores más altos para el ratio disminuían los indicadores del modelo, al tiempo que ratios más bajos aumentaban la precisión y las F, a costa de una menor sensibilidad a fraudes. Comparando con el oversampling, con el SMOTE obtuvo unos indicadores beta ligeramente más altos a costa de una precisión menor.

# Exercice 04.5

Estimate a Logistic Regression, GaussianNB, K-nearest neighbors and a Decision Tree **Classifiers**

Evaluate using the following metrics:
* Accuracy
* F1-Score
* F_Beta-Score (Beta=10)

Comment about the results

Combine the classifiers and comment

In [210]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
models = {'lr': LogisticRegression(),
          'dt': DecisionTreeClassifier(),
          'nb': GaussianNB(),
          'nn': KNeighborsClassifier()}

for model in models.keys():
    models[model].fit(X_train, y_train)
    
y_pred = pd.DataFrame(index=y_test.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)
    
for model in models.keys():
    print(model,metrics.accuracy_score(y_test,y_pred[model]),metrics.f1_score(y_test,y_pred[model]),metrics.fbeta_score(y_test,y_pred[model],beta=10))

lr 0.9939736455119518 0.0 0.0
dt 0.9888411522159107 0.14190687361419066 0.15287106234036513
nb 0.9913785646319311 0.0 0.0
nn 0.9937429716559499 0.1422924901185771 0.08680290297937356


C:\Users\AndresGarcia\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


De acuerdo con lo anterior, el modelo con mejores puntajes en general es el árbol de decisiones, debido a que es el modelo con el mejor puntaje F-beta, el segundo mejor F1 (con muy poca diferencia), y muy buena precisión.

Sin embargo, en general se observa que los modelos presentan mpuntajes bajos, lo cual habla de su poca especificidad. Lo anterior va ligado al uso de la muestra desbalanceada original.

Al mismo tiempo, se aprecia que el modelo de árboles de decisión obtuvo mejores resultados que el modelo de regresión logística original, haciéndolo más adecuado para la predicción de fraudes dada la información disponible.

A continuación se combinan los valores obtenidos, redondeando hacia arriba. En este caso, si uno de los modelos clasificaba la transacción como fraude, se redondeaba la variable dependiente hacia arriba, para indicar fraude.

In [225]:
print("Accuracy",metrics.accuracy_score(y_test,np.ceil(y_pred.mean(axis=1))))
print("F1 Score",metrics.f1_score(y_test,np.ceil(y_pred.mean(axis=1))))
print("F-Beta",metrics.fbeta_score(y_test,np.ceil(y_pred.mean(axis=1)),beta=10))

confusion_matrix(y_test, np.ceil(y_pred.mean(axis=1)))

Accuracy 0.9859288947838875
F1 Score 0.13780918727915192
F-Beta 0.18530366467516582


array([[34154,   318],
       [  170,    39]], dtype=int64)

Se observan los problemas con la combinación. Al tiempo que el F-Beta mejoró frente al modelo de árboles de decisión, los puntajes para la prueba F1 y la precisión empeoraron. Lo anterior se observa también en la matriz de confusión: al modelo le falta sensibilidad, detectando apenas 39 de los 209 fraudes en la muestra de pruebas. Nuevamente, esto se ve influenciado por el uso de la base origina, sin técnicas de rebalanceo de clases.

# Exercice 04.6

Using the under-sampled dataset

Evaluate a RandomForestClassifier and compare the results

change n_estimators=100, what happened

Para este ejercicio, se utilizó el AUC como criterio para seleccionar el número óptimo de estimadores. Se buscó maximizar el mismo.

In [259]:
from sklearn.ensemble import RandomForestClassifier
rfclas = RandomForestClassifier()
estimator_range = range(1, 201, 5)

# list to store the average RMSE for each value of n_estimators
AUC_scores = []

import sklearn.metrics as metrics

for estimator in estimator_range:
    rfclas = RandomForestClassifier(n_estimators=estimator, random_state=1, n_jobs=-1)
    rfclas.fit(X_u,y_u)
    AUC_scores.append(metrics.roc_auc_score(y_test,rfclas.predict(X_test)))
    
print()
print("Para este ejercicio se utilizó un bucle, con la finalidad de cambiar el número de estimadores definido en un rango, entre 5 y 200, con saltos de cinco en cinco para dicho número. Así, se obtuvo un AUC máximo de "+str(max(AUC_scores))+", con "+str((AUC_scores.index(max(AUC_scores))+1)*5)+" estimadores")


Para este ejercicio se utilizó un bucle, con la finalidad de cambiar el número de estimadores definido en un rango, entre 5 y 200, con saltos de cinco en cinco para dicho número. Así, se obtuvo un AUC máximo de 0.75202306899657, con 150 estimadores


Para ver el AUC con un número de estimadores igual a 100, se extrajo el valor asociado a 100 estimadores en el vector de AUC, el cual está almacenado como el número 20 en el vector. Dado que Python almacena la primera posición del vector como un cero, el AUC para el número de estimadores 100 estará en la posición 19. 

In [260]:
AUC_scores[19]

0.745672793452227

De acuerdo con lo anterior, definir un número de estimadores igual a 100 disminuye ligeramente el AUC del modelo, frente a los 150 estimadores hallados como óptimos.

Finalmente, se estimó la matriz de confusión y los indicadores de precisión para el modelo random forest con 150 estimadores.

In [263]:
rfclasfinal = RandomForestClassifier(n_estimators=150, random_state=1, n_jobs=-1)
rfclasfinal.fit(X_u,y_u)
y_pred_rf=rfclasfinal.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred_rf))
print(metrics.f1_score(y_test,y_pred_rf))
print(metrics.fbeta_score(y_test,y_pred_rf,beta=10))

confusion_matrix(y_test, y_pred_rf)

0.8470920677027768
0.048789237668161435
0.5229574354679052


array([[29242,  5230],
       [   73,   136]], dtype=int64)

Comparando con el modelo de regresión logística hallado utilizando esta misma muestra, el modelo random forest obtuvo resultados mucho mejores en la clasificación. La precisión, el F1 y el F-beta fueron más altos, a su vez que aumentó la sensibilidad del modelo. Igualmente, se observa una menor cantidad de falsos positivos.